In [1]:
%config Completer.use_jedi = False

In [2]:
from sentence_transformers import SentenceTransformer
data = ["Hello mehmet how are you today","I'm fine and you", "Today it is rainy"]

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
from typing import List, Union, Tuple
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
model=SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')
# embeddings = model.encode(list_data, show_progress_bar=True)

In [5]:
def extract_keywords(docs: List[str],
                     keyphrase_ngram_range: Tuple[int, int] = (1, 1),
                     top_n: int = 5,
                     min_df: int = 1,
                     diversity: float = 0.5,
                     nr_candidates: int = 20,
                     vectorizer: CountVectorizer = None):

        return _extract_keywords_multiple_docs(docs,
                                               keyphrase_ngram_range,
                                               stop_words,
                                               top_n,
                                               min_df,
                                               vectorizer)

In [17]:
def _extract_keywords_multiple_docs(docs: List[str],
                                    keyphrase_ngram_range: Tuple[int, int] = (1, 1),
                                    top_n: int = 5,
                                    min_df: int = 1,
                                    vectorizer: CountVectorizer = None):

    # Extract words
    global count
    if vectorizer:
        count = vectorizer.fit(docs)
    else:
        count = CountVectorizer().fit(docs)
    global words
    global df
    words = count.get_feature_names()
    df = count.transform(docs)

    # Extract embeddings
    global word_embeddings
    global doc_embeddings
    word_embeddings = _extract_embeddings(words)
    doc_embeddings = _extract_embeddings(docs)
    # word_embeddings = self.model.encode(words, show_progress_bar=True)
    # doc_embeddings = self.model.encode(docs, show_progress_bar=True)
    global doc_word_embeddings
    global distances
    global doc_keywords
    # Extract keywords
    global docss
    docss = docs
    keywords = []
    for index, doc in enumerate(docs):
        doc_words = [words[i] for i in df[index].nonzero()[1]]
        if doc_words:
            doc_word_embeddings = np.array([word_embeddings[i] for i in df[index].nonzero()[1]])
            distances = cosine_similarity([doc_embeddings[index]], doc_word_embeddings)[0]
            doc_keywords = [(doc_words[i], round(float(distances[i]), 4)) for i in distances.argsort()[-top_n:]]
            keywords.append(doc_keywords[::-1])
        else:
            keywords.append(["None Found"])

    return keywords


In [18]:
def _extract_embeddings(documents: Union[List[str], str]) -> np.ndarray:
    
    embeddings = model.encode(documents)

    return embeddings

In [19]:
keys = _extract_keywords_multiple_docs(data)
for i in keys:
    print(i)

[('hello', 0.4542), ('mehmet', 0.3992), ('today', 0.3238), ('you', 0.3041), ('how', 0.2213)]
[('fine', 0.6716), ('you', 0.4057), ('and', 0.2061)]
[('rainy', 0.7416), ('today', 0.4608), ('it', 0.2287), ('is', 0.2111)]


In [12]:
count

CountVectorizer()

In [13]:
words

['and',
 'are',
 'fine',
 'hello',
 'how',
 'is',
 'it',
 'mehmet',
 'rainy',
 'today',
 'you']

In [14]:
df.toarray()

array([[0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0]], dtype=int64)

In [22]:
doc_word_embeddings[1]

array([ 1.12196334e-01, -4.97781374e-02,  2.29725957e-01, -7.49675259e-02,
       -3.62224013e-01, -2.27448389e-01, -4.69431549e-01,  4.40572262e-01,
       -5.74928112e-02, -9.71604884e-02,  1.10986583e-01, -4.09315675e-01,
        7.44651556e-02, -2.84056693e-01,  4.51938391e-01, -3.80099893e-01,
        1.16217948e-01, -2.37163141e-01, -2.73544401e-01, -4.31471854e-01,
       -3.63283068e-01, -4.36051637e-01,  2.45205715e-01,  1.86396196e-01,
        1.19629897e-01, -6.53559938e-02,  3.23349953e-01,  5.46111345e-01,
        1.30392253e-01,  2.32021913e-01, -5.19764833e-02,  4.63346904e-03,
       -3.51227149e-02, -8.32454488e-02, -1.27970248e-01, -1.46831349e-02,
        3.59459728e-01, -1.28858507e-01, -1.57206565e-01,  1.05993003e-02,
       -3.31086546e-01, -9.55372229e-02, -3.01532652e-02,  1.36525407e-01,
        1.43311424e-02,  1.77170038e-01, -1.33415341e-01, -9.87558886e-02,
        3.20549123e-02, -2.50094235e-02,  2.85693377e-01,  8.56442004e-03,
        1.74827412e-01,  

In [54]:
distances

array([[0.21635672, 0.4542396 , 0.22132626, 0.3991895 , 0.32384202,
        0.3040914 ]], dtype=float32)

In [42]:
doc_keywords

[('is', 0.2111), ('it', 0.2287), ('today', 0.4608), ('rainy', 0.7416)]

In [47]:
distances.argsort()[-5:]

array([0, 1, 3, 2], dtype=int64)

In [48]:
distances.argsort()

array([0, 1, 3, 2], dtype=int64)

In [21]:
docss

['Hello mehmet how are you today', "I'm fine and you", 'Today it is rainy']

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
stopwords = nlp.Defaults.stop_words
len(stopwords)

326

In [3]:
from top2vec import Top2Vec
model = Top2Vec(documents)

NameError: name 'documents' is not defined

In [4]:
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

iris = load_iris()
X = iris.data

sil_score_max = -1 #this is the minimum possible score

for n_clusters in range(2,10):
    model = KMeans(n_clusters = n_clusters, init='k-means++', max_iter=100, n_init=1)
    labels = model.fit_predict(X)
    sil_score = silhouette_score(X, labels)
    print("The average silhouette score for %i clusters is %0.2f" %(n_clusters,sil_score))
    if sil_score > sil_score_max:
        sil_score_max = sil_score
        best_n_clusters = n_clusters
best_n_clusters

The average silhouette score for 2 clusters is 0.68
The average silhouette score for 3 clusters is 0.55
The average silhouette score for 4 clusters is 0.50
The average silhouette score for 5 clusters is 0.49
The average silhouette score for 6 clusters is 0.48
The average silhouette score for 7 clusters is 0.34
The average silhouette score for 8 clusters is 0.28
The average silhouette score for 9 clusters is 0.32


2

In [6]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

sentences = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.'
          ]

#Encode all sentences
embeddings = model.encode(sentences)

#Compute cosine similarity between all pairs
cos_sim = util.pytorch_cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(i, j, cos_sim[i][j]))

Top-5 most similar pairs:
A man is riding a horse. 	 A man is riding a white horse on an enclosed ground. 	 0.8489
A man is eating food. 	 A man is eating a piece of bread. 	 0.7784
A monkey is playing drums. 	 Someone in a gorilla costume is playing a set of drums. 	 0.7041
A man is eating food. 	 A man is riding a horse. 	 0.3604
A man is eating food. 	 A man is riding a white horse on an enclosed ground. 	 0.3573


In [10]:
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(i, j, cos_sim[i][j]))

3 	 6 	 0.8489
0 	 1 	 0.7784
7 	 8 	 0.7041
0 	 3 	 0.3604
0 	 6 	 0.3573


In [1]:
import pickle
import os
from os.path import exists
# other imports required for nltk ...

myfile = "clf_NUMB.pickle"

objects = []
with (open(myfile, "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile,))
        except EOFError:
            break
#     pickle.dump(classifier, f)

ModuleNotFoundError: No module named 'cPickle'

In [2]:
import pickle
myfile = "clf_NUMB.pickle"

objects = []
with (open(myfile, "rb")) as openfile:
    data = pickle.load(openfile,fix_imports=True)


ModuleNotFoundError: No module named 'sklearn'

In [1]:
import normalise

ModuleNotFoundError: No module named 'sklearn'

In [1]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

sentences = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.'
          ]

#Encode all sentences
embeddings = model.encode(sentences)

#Compute cosine similarity between all pairs
cos_sim = util.pytorch_cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))

Top-5 most similar pairs:
A man is riding a horse. 	 A man is riding a white horse on an enclosed ground. 	 0.8289
A man is eating food. 	 A man is eating a piece of bread. 	 0.7977
A monkey is playing drums. 	 Someone in a gorilla costume is playing a set of drums. 	 0.6842
A woman is playing violin. 	 Someone in a gorilla costume is playing a set of drums. 	 0.3762
A man is eating food. 	 A man is riding a horse. 	 0.3545


In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.4579
A man is playing guitar 		 A woman watches TV 		 Score: 0.1759
The new movie is awesome 		 The new movie is so great 		 Score: 0.9283


In [10]:
cosine_scores

tensor([[ 0.4579,  0.1059,  0.1447],
        [ 0.1239,  0.1759, -0.0344],
        [ 0.1696,  0.1313,  0.9283]])

In [12]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Single list of sentences - Possible tens of thousands of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

paraphrases = util.paraphrase_mining(model, sentences)

for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

  2%|█▋                                                                             | 5.09M/245M [00:20<08:03, 496kB/s]

The new movie is awesome 		 The new movie is so great 		 Score: 0.9283
The cat sits outside 		 The cat plays in the garden 		 Score: 0.6855
I love pasta 		 Do you like pizza? 		 Score: 0.5420
I love pasta 		 The new movie is awesome 		 Score: 0.2629
I love pasta 		 The new movie is so great 		 Score: 0.2268
The new movie is awesome 		 Do you like pizza? 		 Score: 0.1885
A man is playing guitar 		 A woman watches TV 		 Score: 0.1759
The new movie is so great 		 Do you like pizza? 		 Score: 0.1615
The cat plays in the garden 		 A woman watches TV 		 Score: 0.1521
The cat sits outside 		 The new movie is awesome 		 Score: 0.1475


In [13]:
"""
This is a simple application for sentence embeddings: semantic search

We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.

This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'A cheetah is running behind its prey.'
          ]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """





Query: A man is eating pasta.

Top 5 most similar sentences in corpus:
A man is eating food. (Score: 0.7096)
A man is eating a piece of bread. (Score: 0.6074)
A man is riding a horse. (Score: 0.3360)
A man is riding a white horse on an enclosed ground. (Score: 0.3069)
A woman is playing violin. (Score: 0.2378)




Query: Someone in a gorilla costume is playing a set of drums.

Top 5 most similar sentences in corpus:
A monkey is playing drums. (Score: 0.6842)
A woman is playing violin. (Score: 0.3762)
A man is riding a horse. (Score: 0.3079)
A cheetah is running behind its prey. (Score: 0.2760)
A man is eating a piece of bread. (Score: 0.2495)




Query: A cheetah chases prey on across a field.

Top 5 most similar sentences in corpus:
A cheetah is running behind its prey. (Score: 0.7814)
A monkey is playing drums. (Score: 0.2824)
A man is riding a white horse on an enclosed ground. (Score: 0.2208)
A man is riding a horse. (Score: 0.2017)
A man is eating food. (Score: 0.1886)


In [22]:
embedder.tokenizer()

TypeError: __call__() missing 1 required positional argument: 'text'

In [15]:
%config Completer.use_jedi = False

In [3]:
from pytopicrank import TopicRank

ModuleNotFoundError: No module named 'sklearn'

In [4]:
!pip install sklearn